In [7]:
import numpy as np
from glob import glob
import tensorflow as tf
import os

from tensorflow.keras.utils import Sequence

In [8]:
FOLDER = "/media/watemerald/Seagate/data/yt8m/video/"

tp = "val"

In [9]:
files = glob(os.path.join(FOLDER, f"{tp}*.tfrecord"))

In [24]:
%%timeit -n 1 -r 1


ids = []
for e in tf.data.TFRecordDataset(
    tf.data.Dataset.list_files(os.path.join(FOLDER, f"{tp}*tfrecord"))
).as_numpy_iterator():
    tf_example = tf.train.Example.FromString(e)
    ids.append(
        tf_example.features.feature["id"]
        .bytes_list.value[0]
        .decode(encoding="UTF-8")
    ) 

2min 31s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [25]:
# Records per file

counts = []
for f in files:
    c = 0
    for e in tf.data.TFRecordDataset(f).as_numpy_iterator():
        c += 1
    counts.append((f, c))
counts

[('/media/watemerald/Seagate/data/yt8m/video/validate0203.tfrecord', 270),
 ('/media/watemerald/Seagate/data/yt8m/video/validate0561.tfrecord', 294),
 ('/media/watemerald/Seagate/data/yt8m/video/validate1016.tfrecord', 276),
 ('/media/watemerald/Seagate/data/yt8m/video/validate1452.tfrecord', 298),
 ('/media/watemerald/Seagate/data/yt8m/video/validate2046.tfrecord', 313),
 ('/media/watemerald/Seagate/data/yt8m/video/validate2703.tfrecord', 272),
 ('/media/watemerald/Seagate/data/yt8m/video/validate3243.tfrecord', 284),
 ('/media/watemerald/Seagate/data/yt8m/video/validate0000.tfrecord', 284),
 ('/media/watemerald/Seagate/data/yt8m/video/validate0001.tfrecord', 305),
 ('/media/watemerald/Seagate/data/yt8m/video/validate0002.tfrecord', 313),
 ('/media/watemerald/Seagate/data/yt8m/video/validate0003.tfrecord', 300),
 ('/media/watemerald/Seagate/data/yt8m/video/validate0004.tfrecord', 284),
 ('/media/watemerald/Seagate/data/yt8m/video/validate0005.tfrecord', 301),
 ('/media/watemerald/Seag

In [113]:
ids = []
for batch in tf.data.TFRecordDataset(tf.data.Dataset.list_files(os.path.join(FOLDER, f"{tp}*tfrecord"))).batch(12):
    tid = []
    for e in batch.numpy():
        tf_example = tf.train.Example.FromString(e)
        tid.append(
            tf_example.features.feature["id"]
            .bytes_list.value[0]
            .decode(encoding="UTF-8")
        )
    ids.append(tid)

In [114]:
ids = np.stack(ids)

In [115]:
ids.shape

(1112356,)

In [116]:
c=0
for e in tf.data.TFRecordDataset(tf.data.Dataset.list_files(os.path.join(FOLDER, f"{tp}*tfrecord"))):
    c+=1
c

1112356

In [22]:
a = tf.data.TFRecordDataset(tf.data.Dataset.list_files(os.path.join(FOLDER, f"{tp}*tfrecord")))

In [301]:
a = tf.data.TFRecordDataset('/media/watemerald/Seagate/data/yt8m/video/validate1006.tfrecord')

In [358]:
b = a.cache().map(parse_example).repeat().shuffle(256).batch(16).prefetch(tf.data.experimental.AUTOTUNE)
b

<PrefetchDataset shapes: ((None,), (None, 1, 128), (None, 1, 1024), (None, None, None)), types: (tf.string, tf.float32, tf.float32, tf.int64)>

In [218]:
d = b.skip(300).take(30).as_numpy_iterator()
c = list(d)

In [208]:
len(c)

30

In [209]:
len(c[0])

16

In [219]:
len(list(d))

0

In [228]:
def tf_itr(
    tp: str = "test",
    batch: int = 1024,
    skip: int = 0,
    *,
    media_folder: str,
    **kwargs,
):
    """
    Iterate over TFRecords of a certain type

    Args:
        tp: the type of record to be iterated over (can be "train", "test", or "validate")
        batch: the number of records to be yielded at once

    Yields:
        (array, array, array, array): a tuple of numpy arrays, (ids, audio, rgb, labels) where for a specific
            index i ids[i], audio[i], rgb[i], labels[i] all correspond to the same records info
    """
    # TFRecord files
    gl = os.path.join(media_folder, f"{tp}*tfrecord")
    tfiles = sorted(glob.glob(gl))

    self.log.info(f"total files in {tp} {len(tfiles)}")

    # Initialize the lists to store the ids, audio & visual information, and labels for the current batch
    for example_batch in (
        tf.data.TFRecordDataset(tf.data.Dataset.list_files(gl))
        .cache()
        .repeat()
        .shuffle(256)
        .batch(batch)
#         .prefetch(tf.data.experimental.AUTOTUNE)
        .skip(skip)
    ):
        ids, aud, rgb, lbs = [], [], [], []
        for example in example_batch:
            # Parse a single record from the dataset and extract all its values
            tf_example = tf.train.Example.FromString(example)

            ids.append(
                tf_example.features.feature["id"]
                .bytes_list.value[0]
                .decode(encoding="UTF-8")
            )
            rgb.append(
                np.array(tf_example.features.feature["mean_rgb"].float_list.value)
            )
            aud.append(
                np.array(tf_example.features.feature["mean_audio"].float_list.value)
            )

            # Convert a list of labels into a 1D vector where all the labels are marked as 1
            yss = np.array(tf_example.features.feature["labels"].int64_list.value)

            out = np.zeros(OUTPUT_CLASSES).astype(np.int8)
            for y in yss:
                out[y] = 1

            lbs.append(out)

        yield np.array(ids), np.array(aud), np.array(rgb), np.array(lbs)


In [229]:
tf_itr("val", 16, 0, media_folder=FOLDER)

<generator object tf_itr at 0x7efa8a0940d0>

In [235]:
tf.train.Example.FromString(e)

features {
  feature {
    key: "id"
    value {
      bytes_list {
        value: "spqo"
      }
    }
  }
  feature {
    key: "labels"
    value {
      int64_list {
        value: 1
        value: 36
        value: 185
      }
    }
  }
  feature {
    key: "mean_audio"
    value {
      float_list {
        value: -0.6728783249855042
        value: -0.23314322531223297
        value: -0.6482581496238708
        value: 0.6944246292114258
        value: -0.532429039478302
        value: -1.399537205696106
        value: 0.45580625534057617
        value: -1.1075232028961182
        value: 0.6650258302688599
        value: 0.8124352693557739
        value: -0.1729951798915863
        value: -0.25755563378334045
        value: 0.9029170274734497
        value: 0.33021217584609985
        value: -0.8767998814582825
        value: -1.3173660039901733
        value: -0.7363505959510803
        value: -0.8017966151237488
        value: 0.6789461374282837
        value: 0.43191325664520264

In [152]:
import tensorflow.keras.backend as K

In [177]:
@tf.function
def to_class_label(labels):
    out = tf.Variable(np.zeros(OUTPUT_CLASSES))
    for i in labels:
        out[i] = 1.0
    return out

In [475]:
OUTPUT_CLASSES = 4716
def parse_example(e):
    features = {
        "id": tf.io.FixedLenFeature(shape=(), dtype=tf.string),
        "labels": tf.io.VarLenFeature(tf.int64),
        "mean_audio": tf.io.FixedLenFeature(shape=(128,), dtype=tf.float32),
        "mean_rgb": tf.io.FixedLenFeature(shape=(1024), dtype=tf.float32),
    }
    example = tf.io.parse_example(
        e[tf.newaxis],
        features
    )

    labels = tf.squeeze(
        tf.sparse.to_dense(
            example["labels"]
        ),
        axis=0
    )
      
    classes = tf.one_hot(labels, OUTPUT_CLASSES)
    z = tf.zeros((1,OUTPUT_CLASSES))
    classes = tf.concat([z, classes], axis=0)
    
    
#     tf.reduce_sum(tf.keras.utils.to_categorical(_32, num_classes=OUTPUT_CLASSES), axis=0)
    return (
        {
            "x1": tf.reshape(example["mean_audio"], (128,)), 
            "x2": tf.reshape(example["mean_rgb"], (1024,))
        }, 
#         tf.reduce_sum(tf.keras.utils.to_categorical(labels, num_classes=OUTPUT_CLASSES), axis=0)
#         tf.keras.utils.to_categorical(labels, num_classes=OUTPUT_CLASSES)
        tf.reduce_sum(classes, axis=0)
    )

In [476]:
i = iter(a)

In [489]:
e = next(i)

In [490]:
tf.train.Example.FromString(e.numpy())

features {
  feature {
    key: "id"
    value {
      bytes_list {
        value: "8tqo"
      }
    }
  }
  feature {
    key: "labels"
    value {
      int64_list {
        value: 5
      }
    }
  }
  feature {
    key: "mean_audio"
    value {
      float_list {
        value: 1.091794729232788
        value: -0.3761981129646301
        value: -1.0859452486038208
        value: 0.8205812573432922
        value: -0.00048531187348999083
        value: -0.038905318826436996
        value: 0.30278265476226807
        value: -0.32807838916778564
        value: -0.14295002818107605
        value: -0.3402788043022156
        value: -1.7425711154937744
        value: -1.1103460788726807
        value: 0.3409753441810608
        value: 0.04755863919854164
        value: 0.213060200214386
        value: 0.920609712600708
        value: 0.14849640429019928
        value: -0.20705914497375488
        value: 0.7545018792152405
        value: -1.0813984870910645
        value: -0.3026165962219

In [491]:
example = parse_example(e)

In [414]:
example

({'x1': <tf.Tensor: shape=(128,), dtype=float32, numpy=
  array([ 0.82158005, -0.84183824, -0.24932796, -0.9750033 , -0.7864314 ,
         -0.19621006,  0.45231304,  1.9186767 ,  0.43400118, -1.1107266 ,
          1.0205197 , -0.01033107, -0.6210186 , -0.7958566 , -0.58883816,
         -0.78165144, -0.5413754 ,  0.63543177, -1.4169118 ,  0.19803382,
          0.40801448,  1.7235745 , -1.0646776 ,  1.2922356 , -0.058871  ,
         -1.6575246 , -1.2582314 ,  1.8882468 ,  0.78394645,  0.26993486,
         -1.0216582 ,  0.5032093 ,  1.2220176 ,  0.9276139 ,  0.15057106,
         -0.49162367, -0.47432163,  0.14518522, -1.8082609 , -0.43709198,
          0.13676983, -0.3700382 ,  1.3142502 , -1.3761141 ,  0.3239953 ,
         -0.52972853,  0.41999796,  0.51660657,  0.44187796,  0.20941143,
         -0.3553618 , -0.0693734 , -0.95648944,  0.55652916,  0.28097585,
         -1.547586  ,  0.32298544, -0.6107182 ,  0.4574296 ,  0.26569352,
         -1.7660493 , -1.0877694 ,  1.6319479 ,  0.37381

In [302]:
tf.sparse.to_dense( example['labels'])

<tf.Tensor: shape=(1, 3), dtype=int64, numpy=array([[  1,  36, 185]])>

In [315]:
parse_example(e)[3]

array([[  1,  36, 185]])

In [360]:
e = next(b.as_numpy_iterator())

In [339]:
tf.pad(tf.convert_to_tensor([1,2,3]), [[0,0]])

<tf.Tensor: shape=(4,), dtype=int32, numpy=array([0, 1, 2, 3], dtype=int32)>

In [364]:
ids, aud, rgb, yss = e

(16, 1, 128)

In [3]:

from tensorflow.keras.initializers import glorot_normal
from tensorflow.keras.layers import (
    BatchNormalization,
    Dense,
    Dropout,
    Input,
    Layer,
    LeakyReLU,
    concatenate,
)
from tensorflow.keras.models import Model

In [4]:
def fc_block(x: Layer, n: int = 1024, d: float = 0.2) -> Layer:
    """
        Passes a TensorFlow layer through a block of layers

        Args:
            x: a TensorFlow layer to be passed through the block
            n: the number of neurons in the dense layer
            d: the dropout rate

        Returns:
            Layer: the resulting Tensorflow Layer
    """
    x = Dense(n, kernel_initializer=glorot_normal())(x)
    x = BatchNormalization()(x)
    x = LeakyReLU()(x)
    x = Dropout(d)(x)
    return x

In [530]:
AUDIO_DATA = 128
VIDEO_DATA = 1024

# Input 1 is the audio information
in1 = Input((AUDIO_DATA,), name="x1")
x1 = fc_block(in1)

# Input 2 is the video information
in2 = Input((VIDEO_DATA,), name="x2")
x2 = fc_block(in2)

x = concatenate([x1, x2], 1)
x = fc_block(x)
out = Dense(OUTPUT_CLASSES, activation="sigmoid", name="output")(x)

model = Model([in1, in2], out)
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=[tf.keras.metrics.TopKCategoricalAccuracy(k=20)])

In [396]:
a = tf.data.TFRecordDataset(tf.data.Dataset.list_files(os.path.join(FOLDER, f"{tp}*tfrecord")))

In [531]:
tp = "train"
youtube_dataset = tf.data.TFRecordDataset(tf.data.Dataset.list_files(os.path.join(FOLDER, f"{tp}*tfrecord")))
youtube_dataset = youtube_dataset.cache().map(parse_example).repeat().shuffle(1024).batch(256).prefetch(tf.data.experimental.AUTOTUNE)

In [517]:
next(iter(youtube_dataset))

({'x1': <tf.Tensor: shape=(256, 128), dtype=float32, numpy=
  array([[-0.47034997,  1.0291682 ,  0.4648209 , ...,  0.47710103,
           0.97897214, -0.3446805 ],
         [-1.5981331 , -1.0116056 , -0.08757017, ..., -0.17276941,
           0.65638936, -0.22084025],
         [ 1.189721  , -0.41901103, -0.43783456, ..., -0.9855993 ,
          -0.19380842, -1.3280829 ],
         ...,
         [ 1.3431066 , -0.5829778 , -0.53894925, ..., -0.06717265,
           0.2517816 ,  0.49046254],
         [-0.92686206,  0.4547547 ,  0.06411398, ..., -0.65838766,
          -0.63325465, -0.79746485],
         [-0.22190887, -0.480182  , -0.12125527, ..., -0.65974855,
          -0.20732337, -0.56397974]], dtype=float32)>,
  'x2': <tf.Tensor: shape=(256, 1024), dtype=float32, numpy=
  array([[ 5.76052181e-02,  8.58143032e-01, -1.55459210e-01, ...,
          -1.23180652e+00,  1.83447134e-02, -1.13150406e+00],
         [ 1.19857788e+00, -1.07747272e-01,  2.62818225e-02, ...,
          -3.77282985e-02,  2

In [ ]:
model.fit(youtube_dataset, epochs=2, steps_per_epoch=15191)

Epoch 1/2
10062/15191 [==================>...........] - ETA: 1:58 - loss: 7.6940 - top_k_categorical_accuracy: 0.9465

In [58]:
tf.squeeze(example[1])

<tf.Tensor: shape=(3,), dtype=int64, numpy=array([  1,  36, 185])>

In [32]:
tf.convert_to_tensor([1,36,185])

<tf.Tensor: shape=(3,), dtype=int32, numpy=array([  1,  36, 185], dtype=int32)>

In [52]:
tf.reduce_sum(tf.keras.utils.to_categorical(_32, num_classes=OUTPUT_CLASSES), axis=0).numpy()[185]

1.0

In [96]:
y = np.array([0,1], dtype='int')

In [124]:
t = example[1]

In [130]:
y = np.zeros(OUTPUT_CLASSES)

In [140]:
y[np.array(t)] = 1

In [141]:

y

array([0., 1., 0., ..., 0., 0., 0.])

In [156]:
dir(K)

['__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 '_sys',
 'abs',
 'all',
 'any',
 'arange',
 'argmax',
 'argmin',
 'backend',
 'batch_dot',
 'batch_flatten',
 'batch_get_value',
 'batch_normalization',
 'batch_set_value',
 'bias_add',
 'binary_crossentropy',
 'cast',
 'cast_to_floatx',
 'categorical_crossentropy',
 'clear_session',
 'clip',
 'concatenate',
 'constant',
 'conv1d',
 'conv2d',
 'conv2d_transpose',
 'conv3d',
 'cos',
 'count_params',
 'ctc_batch_cost',
 'ctc_decode',
 'ctc_label_dense_to_sparse',
 'cumprod',
 'cumsum',
 'depthwise_conv2d',
 'dot',
 'dropout',
 'dtype',
 'elu',
 'epsilon',
 'equal',
 'eval',
 'exp',
 'expand_dims',
 'eye',
 'flatten',
 'floatx',
 'foldl',
 'foldr',
 'function',
 'gather',
 'get_uid',
 'get_value',
 'gradients',
 'greater',
 'greater_equal',
 'hard_sigmoid',
 'image_data_format',
 'in_test_phase',
 'in_top_k',
 'in_train_phase',
 'int_shape',
 'is_keras_tensor',
 

In [257]:
t = example[1]

In [199]:
_194[185]

<tf.Tensor: shape=(), dtype=float32, numpy=1.0>

In [260]:

tf.one_hot(t, OUTPUT_CLASSES)

<tf.Tensor: shape=(3, 4716), dtype=float32, numpy=
array([[0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)>

In [261]:
tf.zeros((1,OUTPUT_CLASSES))

<tf.Tensor: shape=(1, 4716), dtype=float32, numpy=array([[0., 0., 0., ..., 0., 0., 0.]], dtype=float32)>

In [274]:
tf.concat([_260, _261], axis=0)

<tf.Tensor: shape=(4, 4716), dtype=float32, numpy=
array([[0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)>

In [276]:
tf.reduce_sum(_274, axis=0)

<tf.Tensor: shape=(4716,), dtype=float32, numpy=array([0., 1., 0., ..., 0., 0., 0.], dtype=float32)>

In [277]:
classes = tf.convert_to_tensor([])

In [281]:
tf.concat([_261, _280], axis=0)

<tf.Tensor: shape=(1, 4716), dtype=float32, numpy=array([[0., 0., 0., ..., 0., 0., 0.]], dtype=float32)>

In [420]:
tf.one_hot([5], OUTPUT_CLASSES)

<tf.Tensor: shape=(1, 4716), dtype=float32, numpy=array([[0., 0., 0., ..., 0., 0., 0.]], dtype=float32)>

In [426]:
_420[0][5]

<tf.Tensor: shape=(), dtype=float32, numpy=1.0>

In [520]:
# Length of training dataset
from video_search.utils import count_records

In [522]:
total_records = count_records(FOLDER, "train")

In [524]:
total_records // 256

15191